# Lesson 3: Sentence Window Retrieval

In [1]:
import warnings
import utils
import os
import openai

warnings.filterwarnings("ignore")


openai.api_key = utils.get_openai_api_key()

/home/chris/anaconda3/envs/llamaindex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: 1b47a87c-4f76-4e62-89d7-3bbd6035c205
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

## Window-sentence retrieval setup

In [5]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [7]:
# Each sentence from the above is split into it's own element.
print([x.text for x in nodes])

# Loop through each node within the nodes list and print the text of each node.
for node in nodes:
    print(node.text)

['hello. ', 'how are you? ', 'I am fine!  ']
hello. 
how are you? 
I am fine!  


In [8]:
print(nodes[0].metadata["window"])
print(nodes[1].metadata["window"])
print(nodes[2].metadata["window"])

hello.  how are you?  I am fine!  
hello.  how are you?  I am fine!  
hello.  how are you?  I am fine!  


In [9]:
text = """
Exploring varied color schemes and patterns in your rooms infuses them with life and personality. Get in touch if you need expert guidance on where to begin. Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.

There is no definitive blueprint for design. The most enchanting spaces are the ones that bring you comfort and joy. Therefore, feel free to experiment, bend the rules, and design a home that is distinctly yours.

May these guidelines inspire you to delve into the world of color and pattern in your home. Enjoy the process of decorating! Remember, if you need further inspiration or tips, you can sign up for my newsletter. Happy designing!
"""

In [10]:
nodes = node_parser.get_nodes_from_documents([Document(text=text)])
# Loop through each node within the nodes list and print the text of each node.
for node in nodes:
    print(node.text)

for i in range(len(nodes)):
    print(f"node {i} = {nodes[i].text}")


Exploring varied color schemes and patterns in your rooms infuses them with life and personality. 
Get in touch if you need expert guidance on where to begin. 
Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.


There is no definitive blueprint for design. 
The most enchanting spaces are the ones that bring you comfort and joy. 
Therefore, feel free to experiment, bend the rules, and design a home that is distinctly yours.


May these guidelines inspire you to delve into the world of color and pattern in your home. 
Enjoy the process of decorating! 
Remember, if you need further inspiration or tips, you can sign up for my newsletter. 
Happy designing!

node 0 = 
Exploring varied color schemes and patterns in your rooms infuses them with life and personality. 
node 1 = Get in touch if you need expert guidance on where to begin. 
node 2 = Your understanding of basic color theory and pattern u

In [11]:
for i in range(len(nodes)):
    print(f"window {i} = {nodes[i].metadata['window']}")

window 0 = 
Exploring varied color schemes and patterns in your rooms infuses them with life and personality.  Get in touch if you need expert guidance on where to begin.  Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.


window 1 = 
Exploring varied color schemes and patterns in your rooms infuses them with life and personality.  Get in touch if you need expert guidance on where to begin.  Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.

 There is no definitive blueprint for design. 
window 2 = 
Exploring varied color schemes and patterns in your rooms infuses them with life and personality.  Get in touch if you need expert guidance on where to begin.  Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your sty

### Building the index

In [12]:
# create the LLM
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [13]:
# Create the Service Context
from llama_index import ServiceContext

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-large-en-v1.5",
    node_parser=node_parser,
)

config.json: 100%|██████████| 779/779 [00:00<00:00, 5.76MB/s]
model.safetensors: 100%|██████████| 1.34G/1.34G [00:22<00:00, 60.3MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 2.48MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 4.17MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 17.3MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 826kB/s]


In [14]:
# Create the VectorStore
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [15]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage


if not os.path.exists("./sentence_index"):
    # If the index doesn't yet exist, create it
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    # If the index already exists, load it from storage
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context,
    )

### Building the postprocessor

In [16]:
# This will pull the context window out for each selected node.
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

In [17]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [18]:
scored_nodes

[NodeWithScore(node=TextNode(id_='f946f63b-3dbb-48fa-a515-5570605ec9cd', embedding=None, metadata={'window': '\nExploring varied color schemes and patterns in your rooms infuses them with life and personality.  Get in touch if you need expert guidance on where to begin.  Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.\n\n', 'original_text': '\nExploring varied color schemes and patterns in your rooms infuses them with life and personality. '}, excluded_embed_metadata_keys=['window', 'original_text'], excluded_llm_metadata_keys=['window', 'original_text'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ebf5a6fd-9993-4c4f-b1ba-9ec7d56ae112', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='a72f3f78499c07bca9853273f1f4359e7c83c28614918f61c6e85288aaf9e5ea'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1631ab4c-e47a-48ba-9a55-bb0498c26a58', node_type=

In [19]:
nodes_old[1].text

'Get in touch if you need expert guidance on where to begin. '

In [20]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [21]:
print(replaced_nodes[1].text)


Exploring varied color schemes and patterns in your rooms infuses them with life and personality.  Get in touch if you need expert guidance on where to begin.  Your understanding of basic color theory and pattern use will help you craft visually captivating, harmonious spaces that are true to your style.

 There is no definitive blueprint for design. 


### Adding a reranker

In [22]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(top_n=2, model="BAAI/bge-reranker-base")

In [23]:
# Exploring re-ranking
# This assumes the orig query is "I want a dog"
# And the original results are "This is a cat" which is RANKED HIGHER (0.6) than "This is a dog" (0.4)
# But the re-ranked results should have "This is a dog" ranked higher

from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

reranked_nodes = rerank.postprocess_nodes(scored_nodes, query_bundle=query)

print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.91827404), ('This is a cat', 0.0014040867)]


### Runing the query engine

In [24]:
# NOTE that the k value here (6) is higher than that used in the re-ranker.  This gives the re-ranker some options.
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

In [25]:
window_response = sentence_window_engine.query(
    "What are the keys to building a career in AI?"
)

In [26]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.

## Putting it all Together

In [27]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [28]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)

config.json: 100%|██████████| 743/743 [00:00<00:00, 5.09MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:01<00:00, 75.6MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 3.20MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 20.3MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 31.3MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 799kB/s]


In [29]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

## TruLens Evaluation
- Gradually increase the sentence window size start with one
- Evaluate app versions with the RAG Triad
- Track experiments to pick the best sentence window size
- Note tradeoff between token usage/cost and context relevance
- note tradeoff between context relevance and groundedness

### Experiments
- Load different questions
- try different sentence-window size
  - 1
  - 3
  - 5
- Check the impact of the different window size on the RAG Triad

In [35]:
eval_questions = []
with open("generated_questions_01_05.text", "r") as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

In the context of job switching, what advantages does working in a startup offer over a big company, particularly for someone looking to transition into an AI role?
Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?
In the context of project selection and execution, explain the difference between the "Ready, Aim, Fire" and "Ready, Fire, Aim" approaches. Provide examples where each approach might be more beneficial.
Discuss the importance of working on projects that complement your career goals. How can these projects serve as stepping stones to larger projects and contribute to technical growth and skill progression?
According to the text, what are some of the key areas of mathematics that are relevant to machine learning and why are they important in the field of AI?


In [32]:
from trulens_eval import Tru


def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [33]:
from utils import get_prebuilt_trulens_recorder

from trulens_eval import Tru

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### Sentence window size = 1

In [36]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=1,
    save_dir="sentence_index_1",
)

In [37]:
sentence_window_engine_1 = get_sentence_window_query_engine(sentence_index_1)

In [38]:
tru_recorder_1 = get_prebuilt_trulens_recorder(
    sentence_window_engine_1, app_id="sentence window engine 1"
)

In [39]:
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

https://learn.deeplearning.ai/building-evaluating-advanced-rag/lesson/4/sentence-window-retrieval
- start at 19:19

In [41]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…